In [1]:
import numpy as np
from docplex.mp.model import Model

# from docplex.mp.callbacks.cb_mixin import ConstraintCallbackMixin
# from docplex.mp.callbacks.cb_mixin import LazyConstraintCallback
import matplotlib.pyplot as plt

In [2]:
import cplex
from cplex.callbacks import UserCutCallback, LazyConstraintCallback

In [3]:
# 需要点と施設候補の座標データ
demand_points = [
    (2, 22),
    (42, 6),
    (48, 50),
    (32, 40),
    (16, 10),
    (10, 34),
    (8, 30),
    (29, 16),
    (44, 34),
    (3, 16),
]
candidate_sites = [
    (36, 32),
    (38, 32),
    (44, 11),
    (26, 50),
    (22, 28),
    (32, 46),
    (20, 37),
    (35, 26),
    (2, 4),
    (18, 1),
]

In [4]:
J = len(candidate_sites)  # 施設候補の数
D = len(demand_points)  # 需要点の数
alpha = 0
beta = 0.1

In [5]:
h_i = np.full(D, 1 / D)

In [6]:
# 既存のリーダーの施設セット J_L を仮定
J_L = {0, 2}  # インデックスとして候補施設の一部を選択
J_F = {1}

In [7]:
# ユークリッド距離の計算
def compute_distances(demand_points, candidate_sites):
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d][j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances


distances = compute_distances(demand_points, candidate_sites)

In [8]:
# w_ij の計算
wij_matrix = np.exp(alpha - beta * distances)

In [ ]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_L)].sum(axis=1)


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_F)].sum(axis=1)


# U_i^L の計算
Ui_L = compute_Ui_L(wij_matrix, J_L)

# U_i^F の計算
Ui_F = compute_Ui_F(wij_matrix, J_F)

print(f"Ui_L: {Ui_L}")
print(f"Ui_F: {Ui_F}")

Ui_L: [0.04191217 0.65298119 0.13477363 0.45219075 0.1118403  0.09019735
 0.07744449 0.38013697 0.5386599  0.04161902]
Ui_F: [0.02384196 0.072036   0.12756541 0.36787944 0.04454436 0.0603778
 0.04945662 0.15949288 0.53128561 0.02131432]


In [16]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))


x = np.random.randint(0, 2, J)  # ランダムなリーダーの施設選択 (0 or 1)
y = np.random.randint(0, 2, J)  # ランダムなフォロワーの施設選択 (0 or 1)

print(f"x: {x}")
print(f"y: {y}")

# L(x, y) の計算
L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x, y)

# 計算結果の表示
print(f"L(x, y): {L_value}")

x: [0 1 1 1 0 1 0 1 0 1]
y: [0 0 0 0 1 1 1 1 0 0]
L(x, y): 0.6416406724061001


In [11]:
# Lazy Constraint Callback 修正
class LazyCallback(LazyConstraintCallback):
    def __init__(self, model, x, theta):
        super().__init__(model)  # 修正：引数は model のみ
        self.x = x
        self.theta = theta

    def invoke(self, context):
        x_hat = {j: context.get_candidate_value(self.x[j]) for j in range(J)}
        theta_hat = context.get_candidate_value(self.theta)

        # ダミーの L(x, y) を計算（本来は separation problem を解く）
        y_hat = np.random.choice([0, 1], size=D)
        L_x_y = np.sum(y_hat)

        if theta_hat > L_x_y:
            context.add_lazy_constraint(
                self.theta <= L_x_y
            )  # 修正：lazy constraint の追加

In [12]:
# CPLEX モデルの作成と実行
def solve_s_cflp():
    mdl = Model("S-CFLP")
    x = mdl.binary_var_dict(range(J), name="x")
    theta = mdl.continuous_var(name="theta")

    mdl.maximize(theta)

    mdl.add_constraint(mdl.sum(x[j] for j in range(J)) <= 5)  # 施設数の制限

    mdl.register_callback(LazyCallback(mdl, x, theta))
    mdl.solve()

    BestSol = {j: x[j].solution_value for j in range(J)}
    theta_LB = theta.solution_value

    return BestSol, theta_LB

In [13]:
# 最適解の計算とプロット
best_solution, best_theta = solve_s_cflp()
print("Best Solution:", best_solution)
print("Best Theta:", best_theta)


# 結果の可視化
def plot_results():
    plt.figure(figsize=(8, 8))
    plt.scatter(*zip(*demand_points), c="blue", marker="o", label="Demand Points")
    plt.scatter(*zip(*candidate_sites), c="red", marker="s", label="Candidate Sites")

    for j, val in best_solution.items():
        if val > 0.5:
            plt.scatter(
                candidate_sites[j][0],
                candidate_sites[j][1],
                c="green",
                marker="s",
                s=100,
                label="Selected Facility" if j == 0 else "",
            )

    plt.legend()
    plt.title("Optimal Facility Location")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.show()


plot_results()

TypeError: __call__() takes 1 positional argument but 2 were given